In [327]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns

from sklearn import datasets 
from sklearn.feature_selection import RFE 
from sklearn.linear_model import LogisticRegression

from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn import preprocessing

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Binarizer

In [328]:
# read lat/lng data into panda dataframe
conn = sqlite3.connect('FPA_FOD_20170508.sqlite')

# df = pd.read_sql_query("SELECT latitude, longitude FROM fires, STATE", conn)
df = pd.read_sql_query("SELECT FIRE_YEAR, STAT_CAUSE_DESCR, LATITUDE, LONGITUDE, STATE, COUNTY, FIPS_NAME, DISCOVERY_DATE, FIRE_SIZE, DISCOVERY_DOY, DISCOVERY_TIME FROM 'Fires'", conn)
df['DATE'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
print(df.head())

FIRE_YEAR STAT_CAUSE_DESCR   LATITUDE   LONGITUDE STATE COUNTY  FIPS_NAME  \
0       2005    Miscellaneous  40.036944 -121.005833    CA     63     Plumas   
1       2004        Lightning  38.933056 -120.404444    CA     61     Placer   
2       2004   Debris Burning  38.984167 -120.735556    CA     17  El Dorado   
3       2004        Lightning  38.559167 -119.913333    CA      3     Alpine   
4       2004        Lightning  38.559167 -119.933056    CA      3     Alpine   

   DISCOVERY_DATE  FIRE_SIZE  DISCOVERY_DOY DISCOVERY_TIME       DATE  
0       2453403.5       0.10             33           1300 2005-02-02  
1       2453137.5       0.25            133           0845 2004-05-12  
2       2453156.5       0.10            152           1921 2004-05-31  
3       2453184.5       0.10            180           1600 2004-06-28  
4       2453184.5       0.10            180           1600 2004-06-28  


In [329]:
df['TIME'] = df['DISCOVERY_TIME'].str[0:2]
df['DATETIME'] = df['DATE'].dt.strftime('%Y-%m-%d') + ' ' +  df['TIME']
print(df)

FIRE_YEAR   STAT_CAUSE_DESCR   LATITUDE   LONGITUDE STATE COUNTY  \
0             2005      Miscellaneous  40.036944 -121.005833    CA     63   
1             2004          Lightning  38.933056 -120.404444    CA     61   
2             2004     Debris Burning  38.984167 -120.735556    CA     17   
3             2004          Lightning  38.559167 -119.913333    CA      3   
4             2004          Lightning  38.559167 -119.933056    CA      3   
...            ...                ...        ...         ...   ...    ...   
1880460       2015  Missing/Undefined  40.481637 -122.389375    CA   None   
1880461       2015      Miscellaneous  37.617619 -120.938570    CA   None   
1880462       2015  Missing/Undefined  37.617619 -120.938570    CA   None   
1880463       2015  Missing/Undefined  37.672235 -120.898356    CA   None   
1880464       2015      Miscellaneous  34.263217 -116.830950    CA   None   

         FIPS_NAME  DISCOVERY_DATE  FIRE_SIZE  DISCOVERY_DOY DISCOVERY_TIME  \
0    

In [330]:
df = df[df['FIRE_YEAR'] >= 2012]
print(df)

FIRE_YEAR   STAT_CAUSE_DESCR   LATITUDE   LONGITUDE STATE COUNTY  \
1563819       2012           Campfire  45.991944 -113.471389    MT    039   
1563820       2012           Campfire  45.946667 -112.366111    MT    043   
1563821       2012           Campfire  45.418611 -111.855833    MT    057   
1563822       2012           Campfire  46.219167 -112.243333    MT    093   
1563823       2012          Lightning  44.942222 -113.458611    MT    001   
...            ...                ...        ...         ...   ...    ...   
1880460       2015  Missing/Undefined  40.481637 -122.389375    CA   None   
1880461       2015      Miscellaneous  37.617619 -120.938570    CA   None   
1880462       2015  Missing/Undefined  37.617619 -120.938570    CA   None   
1880463       2015  Missing/Undefined  37.672235 -120.898356    CA   None   
1880464       2015      Miscellaneous  34.263217 -116.830950    CA   None   

          FIPS_NAME  DISCOVERY_DATE  FIRE_SIZE  DISCOVERY_DOY DISCOVERY_TIME  \
1563

In [331]:
# df_fire = pd.read_csv('California_Fire_Incidents.csv')
df_humidity = pd.read_csv('humidity.csv')
df_pressure = pd.read_csv('pressure.csv')
df_temp = pd.read_csv('temperature.csv')
df_weather = pd.read_csv('weather_description.csv')
df_wind_direction = pd.read_csv('wind_direction.csv')
df_wind_speed = pd.read_csv('wind_speed.csv')
df_city_attributes = pd.read_csv('city_attributes.csv')

In [332]:
cities = df_humidity.columns.drop(['Jerusalem', 'Haifa', 'Eilat', 'Tel Aviv District', 'Beersheba', 'Nahariyya'])
df_humidity = df_humidity[cities]
df_pressure = df_pressure[cities]
df_temp = df_temp[cities]
df_weather = df_weather[cities]
df_wind_direction = df_wind_direction[cities]
df_wind_speed = df_wind_speed[cities]
# df_city_attributes = df_city_attributes[cities]

In [333]:
print(df_humidity)

datetime  Vancouver  Portland  San Francisco  Seattle  \
0      2012-10-01 13:00:00       76.0      81.0           88.0     81.0   
1      2012-10-01 14:00:00       76.0      80.0           87.0     80.0   
2      2012-10-01 15:00:00       76.0      80.0           86.0     80.0   
3      2012-10-01 16:00:00       77.0      80.0           85.0     79.0   
4      2012-10-01 17:00:00       78.0      79.0           84.0     79.0   
...                    ...        ...       ...            ...      ...   
45247  2017-11-29 20:00:00        NaN      81.0            NaN     93.0   
45248  2017-11-29 21:00:00        NaN      71.0            NaN     87.0   
45249  2017-11-29 22:00:00        NaN      71.0            NaN     93.0   
45250  2017-11-29 23:00:00        NaN      71.0            NaN     87.0   
45251  2017-11-30 00:00:00        NaN      76.0            NaN     75.0   

       Los Angeles  San Diego  Las Vegas  Phoenix  Albuquerque  ...  Detroit  \
0             88.0       82.0       2

In [334]:
common_cities = []
for city in cities:
    if(len(df.loc[df['FIPS_NAME']==city]) != 0):
        common_cities.append(city)

In [335]:
df = df[df['FIPS_NAME'].isin(common_cities)]
print(df)

FIRE_YEAR   STAT_CAUSE_DESCR   LATITUDE   LONGITUDE STATE  \
1567432       2012      Miscellaneous  34.204167 -117.808333    CA   
1567433       2012          Lightning  34.345278 -117.928889    CA   
1567434       2012      Miscellaneous  34.548056 -118.671667    CA   
1567436       2012      Equipment Use  34.489444 -118.285833    CA   
1567437       2012           Children  34.467778 -118.530833    CA   
...            ...                ...        ...         ...   ...   
1872185       2015  Missing/Undefined  33.387140 -117.174866    CA   
1872229       2015     Debris Burning  33.226976 -117.024311    CA   
1872237       2015  Missing/Undefined  34.666666 -118.766666    CA   
1872244       2015  Missing/Undefined  33.388859 -117.255707    CA   
1872247       2015     Debris Burning  33.243255 -117.241177    CA   

              COUNTY    FIPS_NAME  DISCOVERY_DATE  FIRE_SIZE  DISCOVERY_DOY  \
1567432          037  Los Angeles       2456104.5       0.10            178   
1567433   

In [336]:
columns = ['time', 'humidity', 'pressure', 'temperature', 'weather', 'wind direction', 'wind speed']
df_weatherdata = pd.DataFrame()
for city in common_cities:
    newdf = pd.DataFrame(pd.concat([df_humidity['datetime'], df_humidity[city], df_pressure[city], df_temp[city], df_weather[city], df_wind_direction[city], df_wind_speed[city]], axis=1, keys=columns))
    newdf['city'] = city
    df_weatherdata = df_weatherdata.append(newdf, ignore_index=True)

In [337]:
df_weatherdata['DATE'] = df_weatherdata['time'].str[0:11]
df_weatherdata['TIME'] = df_weatherdata['time'].str.split(":").str[0].str[11:]
df_weatherdata['DATETIME'] = df_weatherdata['DATE'] + df_weatherdata['TIME']
print(df_weatherdata['DATETIME'])

0         2012-10-01 13
1         2012-10-01 14
2         2012-10-01 15
3         2012-10-01 16
4         2012-10-01 17
              ...      
362011    2017-11-29 20
362012    2017-11-29 21
362013    2017-11-29 22
362014    2017-11-29 23
362015    2017-11-30 00
Name: DATETIME, Length: 362016, dtype: object


In [338]:
df_weatherdata = df_weatherdata[~(df_weatherdata['DATE'].astype(str).str.contains('2016') | df_weatherdata['DATE'].astype(str).str.contains('2017'))]
print(df_weatherdata['DATE'])

0         2012-10-01 
1         2012-10-01 
2         2012-10-01 
3         2012-10-01 
4         2012-10-01 
             ...     
345234    2015-12-31 
345235    2015-12-31 
345236    2015-12-31 
345237    2015-12-31 
345238    2015-12-31 
Name: DATE, Length: 227800, dtype: object


In [339]:
df_weatherdata['DATETIME_CITY'] = df_weatherdata['DATETIME'] + ' ' + df_weatherdata['city']
print(df_weatherdata['DATETIME_CITY'])

0         2012-10-01 13 San Francisco
1         2012-10-01 14 San Francisco
2         2012-10-01 15 San Francisco
3         2012-10-01 16 San Francisco
4         2012-10-01 17 San Francisco
                     ...             
345234         2015-12-31 19 New York
345235         2015-12-31 20 New York
345236         2015-12-31 21 New York
345237         2015-12-31 22 New York
345238         2015-12-31 23 New York
Name: DATETIME_CITY, Length: 227800, dtype: object


In [340]:
df['DATETIME_CITY'] = df['DATETIME'] + ' ' + df['FIPS_NAME']
print(df['DATETIME_CITY'])

1567432    2012-06-26 10 Los Angeles
1567433    2012-09-10 10 Los Angeles
1567434    2012-11-05 13 Los Angeles
1567436    2012-06-28 10 Los Angeles
1567437    2012-06-21 18 Los Angeles
                     ...            
1872185      2015-11-28 15 San Diego
1872229      2015-12-29 17 San Diego
1872237    2015-11-04 14 Los Angeles
1872244      2015-10-13 03 San Diego
1872247      2015-12-09 16 San Diego
Name: DATETIME_CITY, Length: 3722, dtype: object


In [341]:
if df_weatherdata['DATETIME'][0] in (df['DATETIME']):
    print('true')
else:
    print('false')
# print(df['DATETIME'])

false


In [342]:
print(df_weatherdata['DATETIME'][1+7340])
print(df['DATETIME'].values[2])
if df_weatherdata['DATETIME'][1+7340] in (df['DATETIME'].values):
    print(np.where(df['DATETIME'].values == (df_weatherdata['DATETIME'][1+7340])))

2013-08-03 10
2012-11-05 13
(array([1179]),)


In [343]:
df_weatherdata['FIRE'] = False

In [344]:
df_weatherdata['FIRE'][df_weatherdata.loc[df_weatherdata['DATETIME_CITY'].isin(df['DATETIME_CITY'].values)].index] = True

In [345]:
df_weatherdata = df_weatherdata[df_weatherdata.columns.drop(['time', 'DATE', 'TIME', 'DATETIME'])]

In [346]:
print(df_weatherdata.loc[df_weatherdata['FIRE']==True])

humidity  pressure  temperature               weather  wind direction  \
14176       69.0    1017.0   294.270000          sky is clear           271.0   
14221        NaN    1014.0   292.180000                  haze           334.0   
20087      100.0    1038.0   282.415000         broken clouds           219.0   
21515       77.0    1030.0   289.181500       overcast clouds           288.0   
24145       87.0    1012.0   289.090000                   fog           290.0   
...          ...       ...          ...                   ...             ...   
344254      45.0    1025.0   282.632909  heavy intensity rain           312.0   
344487      60.0    1030.0   276.642214         broken clouds            25.0   
344558      66.0    1011.0   281.390000      scattered clouds           270.0   
344607      65.0    1039.0   280.294636          sky is clear            15.0   
344995      53.0    1025.0   283.583269       overcast clouds           223.0   

        wind speed           city  